<a href="https://colab.research.google.com/github/cccg8105/notebook_AD/blob/deep_learning/deep_learning/red_neuronal/RNA_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Red Neuronal Artificial (RNA)

## Instalar dependendias

In [ ]:
!pip install sklearn
!pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git
!pip install keras
!pip install tensorflow

## Importar conjunto de datos

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

dataset = pd.read_csv('https://raw.githubusercontent.com/cccg8105/deeplearning-az/master/datasets/Part%201%20-%20Artificial%20Neural%20Networks%20(ANN)/Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

## Preprocesado de datos

### Conversión de datos categóricos

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [1])],   
    remainder='passthrough'                        
)
X = onehotencoder.fit_transform(X)
X = X[:, 1:]

array([[0.0, 0.0, 228, ..., 1, 1, 101348.88],
       [1.0, 0.0, 217, ..., 0, 1, 112542.58],
       [0.0, 0.0, 111, ..., 1, 0, 113931.57],
       ...,
       [0.0, 0.0, 318, ..., 0, 1, 42085.58],
       [0.0, 1.0, 381, ..., 1, 0, 92888.52],
       [0.0, 0.0, 401, ..., 1, 0, 38190.78]], dtype=object)

### Dividir en entrenamiento y prueba

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Escalado de variables

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

## Construcción

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

classifier = Sequential()

# Añadir las capas de entrada y primera capa oculta
nodosIniciales = 6 # se recomienda poner la media entre los nodos de entrada y salida (11+1)/2
inicializadorKernel = "uniform" # glorot_uniform|||
funcionActivacion = "relu" # funcion escalon|funcion sigmoide(sigmoid)|tangente hiperbolica(tanh)| rectificador lineal(relu)
nodosEntrada = 11 # numero de variables en la matriz de caracteristicas

classifier.add(Dense(units = nodosIniciales, kernel_initializer = inicializadorKernel,  
                     activation = funcionActivacion, input_dim = nodosEntrada))

# Añadir la segunda capa oculta
nodosCapa2 = 6
inicializadorKernel2 = "uniform" 
funcionActivacion2 = "relu" 
classifier.add(Dense(units = nodosCapa2, kernel_initializer = inicializadorKernel2,  activation = funcionActivacion2))

# Añadir la capa de salida
nodoSalida = 1
salidaKernel = "uniform" 
funcionActivacionSalida = "sigmoid" 
classifier.add(Dense(units = nodoSalida, kernel_initializer = salidaKernel,  activation = funcionActivacionSalida))

## Compilación

In [ ]:
optimizador = "adam" # |||
funcionPerdida =  "binary_crossentropy" # |||
metricas = ["accuracy"] # |||
classifier.compile(optimizer = optimizador, loss = funcionPerdida, metrics = metricas)

## Entrenamiento

In [ ]:
tamanioLote = 10
epocas = 100
classifier.fit(X_train, y_train,  batch_size = tamanioLote, epochs = epocas)

Epoch 1/100
800/800 [==============================] - 1s 854us/step - loss: 0.4855 - accuracy: 0.7958
Epoch 2/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4284 - accuracy: 0.7960
Epoch 3/100
800/800 [==============================] - 1s 858us/step - loss: 0.4238 - accuracy: 0.7960
Epoch 4/100
800/800 [==============================] - 1s 892us/step - loss: 0.4194 - accuracy: 0.8177
Epoch 5/100
800/800 [==============================] - 1s 872us/step - loss: 0.4169 - accuracy: 0.8242
Epoch 6/100
800/800 [==============================] - 1s 852us/step - loss: 0.4153 - accuracy: 0.8275
Epoch 7/100
800/800 [==============================] - 1s 870us/step - loss: 0.4135 - accuracy: 0.8307
Epoch 8/100
800/800 [==============================] - 1s 872us/step - loss: 0.4118 - accuracy: 0.8313
Epoch 9/100
800/800 [==============================] - 1s 867us/step - loss: 0.4113 - accuracy: 0.8324
Epoch 10/100
800/800 [==============================] - 1s 865us/step - los